# Naive Bayes Algorithm

In [41]:
import numpy as np
import pandas as pd

In [42]:
df_train = pd.read_csv('Mushroom_Train.csv')
df_test = pd.read_csv('Mushroom_Test.csv')

In [43]:
X_train=df_train.drop(['class'], axis = 1)
y_train=df_train['class']
X_test=df_test.drop(['class'], axis = 1)
y_test=df_test['class']

In [44]:
featureDict = {}
count=0
for col_name in X_train.columns: 
    featureDict[count]= np.unique(X_train[col_name])
    count=count+1

In [45]:
def calculate_confusion(actual, predicted):

    
    classes = np.unique(actual)
    confusion_matrix = np.zeros((len(classes), len(classes)))
    for i in range(len(classes)):
        for j in range(len(classes)):
            confusion_matrix[i, j] = np.sum((actual == classes[i]) & (predicted == classes[j]))
    
    return confusion_matrix

In [46]:
def Prior_Probability_Calculator(df_train):
    target=df_train["class"]
    e_count=sum(target == 'e')
    p_count=sum(target == 'p')
    return e_count/target.size,p_count/target.size   

In [47]:
def Likelihood_Probability_Calculator(X_train,y_train):
    
    
    e_count=sum(y_train == 'e')
    p_count=sum(y_train == 'p')
    
    train_e = X_train[y_train=='e']
    train_p = X_train[y_train=='p']
    

    dictionary_probE = {}
    dictionary_probP = {}
    cnt=0
    for col_name in train_e.columns: 

        e_unique, e_counts = np.unique(train_e[col_name],return_counts=True)
        p_unique, p_counts = np.unique(train_p[col_name],return_counts=True)
        e_countDict = dict(zip(e_unique,e_counts))
        p_countDict = dict(zip(p_unique,p_counts))
        prob_e = {}
        prob_p = {}
        for st in featureDict[cnt]:
            if st in e_countDict:
                prob_e[st]=float(e_countDict[st]/e_count)
            else:
                prob_e[st]=float(0)
                
            if st in p_countDict:
                prob_p[st]=float(p_countDict[st]/p_count)
            else:
                prob_p[st]=float(0)
               
      
               
        dictionary_probE[cnt]=prob_e
        dictionary_probP[cnt]=prob_p
        cnt=cnt+1
    return dictionary_probE,dictionary_probP

In [48]:
def predict(X_test,e_prior,p_prior):
    
    
    length,d=np.shape(X_test)
    
    stored_data = np.zeros((length,2), dtype=float)

    
    cnt2=0
    for idx,row in X_test.iterrows():
        eProb=e_prior
        pProb=p_prior
        for j in range(0,d):
            
            featuree=row[j]
            if featuree !='?' :
                eProb = eProb * dictionary_probE[j][featuree]
                pProb = pProb * dictionary_probP[j][featuree]
        stored_data[cnt2][0]=eProb
        stored_data[cnt2][1]=pProb
        cnt2=cnt2+1
        
    predicted = np.zeros((length), dtype='str')
    for i in range(0,length):
        if np.argmax(stored_data[i]) == 0:
            predicted[i]='e'
        else:
            predicted[i]='p'
    return predicted

In [49]:
e_prior,p_prior=Prior_Probability_Calculator(df_train)
dictionary_probE,dictionary_probP=Likelihood_Probability_Calculator(X_train,y_train)
y_test_pred=predict(X_test,e_prior,p_prior)
L,dd=np.shape(X_test)

print ("Accuracy percent is:")
print ("%",(y_test_pred==y_test).sum()*100/L)

Accuracy percent is:
% 99.23664122137404


In [50]:
confusion_matrix=calculate_confusion(y_test,y_test_pred)

print("confusion Matrix is:\n")
print(confusion_matrix.astype(int))

confusion Matrix is:

[[245   4]
 [  0 275]]
